# 225229141
# SURUTHI S
# PDL LAB 12
# Image corpus creation and Transfer Learning in CNN

In [ ]:
import datetime
import keras
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
now = datetime.datetime.now  #get current time

batch_size = 128
num_classes = 5
epochs = 5

img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [ ]:
#data, shuffled and split between train and test sets

(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
#create 2 datasets: one with digits below 5 and one with 5 and above

x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [ ]:
x_train_lt5.shape

(30596, 28, 28)

In [ ]:
x_train_gte5.shape

(29404, 28, 28)

In [ ]:
feature_layers =[
    Conv2D(filters, kernel_size = (3,3), activation = 'relu',padding='same',input_shape = (img_rows,img_cols, 1)),
    Conv2D(filters, kernel_size = (3,3), activation = 'relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten()
]

In [ ]:
classification_layers = [
    Dense(128, activation = 'relu'),
    Dropout(0.5),
    Dense(num_classes, activation = 'softmax')
]

In [ ]:
model = Sequential(feature_layers+classification_layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                        

In [ ]:
def train_model(model,train,test,num_classes):
    train[0] = train[0].reshape(train[0].shape[0],img_rows,img_cols,1)
    test[0] = test[0].reshape(test[0].shape[0],img_rows,img_cols,1)

    train[0] = train[0].astype('float32')
    test[0] = test[0].astype('float32')

    train[0] /= 255
    test[0] /= 255

    print(train[0].shape)
    print(test[0].shape)

    s = pd.Series(train[1])
    train[1] = pd.get_dummies(s)
    train[1] = train[1].values

    s = pd.Series(test[1])
    test[1] = pd.get_dummies(s)
    test[1] = test[1].values

    model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])

    train_start_time=now()

    model.fit(train[0],train[1],epochs=epochs,verbose=2)

    print('\n')
    print('\n')

    print('Training time: %s' % (now() - train_start_time))

    score=model.evaluate(test[0],test[1],verbose=0)

    print('test loss ',score[0])
    print('test accuracy ',score[1])

In [ ]:
train_model(model, [x_train_gte5, y_train_gte5], [x_test_gte5, y_test_gte5], num_classes)


(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 14s - loss: 1.5870 - accuracy: 0.2487 - 14s/epoch - 15ms/step
Epoch 2/5
919/919 - 3s - loss: 1.5152 - accuracy: 0.3974 - 3s/epoch - 3ms/step
Epoch 3/5
919/919 - 3s - loss: 1.4247 - accuracy: 0.5478 - 3s/epoch - 4ms/step
Epoch 4/5
919/919 - 3s - loss: 1.3074 - accuracy: 0.6476 - 3s/epoch - 3ms/step
Epoch 5/5
919/919 - 3s - loss: 1.1647 - accuracy: 0.7144 - 3s/epoch - 3ms/step




Training time: 0:00:26.397010
test loss  1.0272140502929688
test accuracy  0.852705180644989


In [ ]:
#Freeze only the feature layers

for l in feature_layers:
  l.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                        

In [ ]:
train_model(model, [x_train_lt5, y_train_lt5], [x_test_lt5, y_test_lt5], num_classes)


(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 4s - loss: 1.4567 - accuracy: 0.4285 - 4s/epoch - 4ms/step
Epoch 2/5
957/957 - 3s - loss: 1.2730 - accuracy: 0.5569 - 3s/epoch - 3ms/step
Epoch 3/5
957/957 - 3s - loss: 1.1306 - accuracy: 0.6707 - 3s/epoch - 3ms/step
Epoch 4/5
957/957 - 3s - loss: 1.0010 - accuracy: 0.7619 - 3s/epoch - 3ms/step
Epoch 5/5
957/957 - 3s - loss: 0.8904 - accuracy: 0.8148 - 3s/epoch - 3ms/step




Training time: 0:00:21.188800
test loss  0.7739014625549316
test accuracy  0.8877213597297668


## Reversing the training process

In [ ]:
modelReverse = Sequential(feature_layers+classification_layers)
modelReverse.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                      

In [ ]:
train_model(modelReverse, [x_train_lt5, y_train_lt5], [x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 4s - loss: 0.7960 - accuracy: 0.8454 - 4s/epoch - 4ms/step
Epoch 2/5
957/957 - 3s - loss: 0.7170 - accuracy: 0.8640 - 3s/epoch - 3ms/step
Epoch 3/5
957/957 - 3s - loss: 0.6528 - accuracy: 0.8770 - 3s/epoch - 3ms/step
Epoch 4/5
957/957 - 3s - loss: 0.6018 - accuracy: 0.8833 - 3s/epoch - 3ms/step
Epoch 5/5
957/957 - 3s - loss: 0.5566 - accuracy: 0.8922 - 3s/epoch - 3ms/step




Training time: 0:00:21.183369
test loss  0.45908495783805847
test accuracy  0.9303366541862488


In [ ]:
#Freeze only the feature layers

for l in feature_layers:
  l.trainable = False

In [ ]:
modelReverse.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                      